<a href="https://colab.research.google.com/github/MarsZDF/bfl-hack-sirius/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sirius Demo 🌟

**AI Image Morphing Pipeline using Claude Vision + FLUX.2**

This notebook demonstrates how to use **Sirius** to create smooth, narrative-driven video transitions between images.

## 1. Setup & Installation
Install the Sirius library and its dependencies.

In [ ]:
# Install sirius directly from GitHub (force reinstall to get latest)
import subprocess
import sys

# Use subprocess for reliable installation in all environments
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "--no-cache-dir", 
                       "git+https://github.com/MarsZDF/bfl-hack-sirius.git"])

# Install optional dependencies
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rich", "nest-asyncio"])

# Clear any cached imports (important for local development)
import importlib
for mod_name in list(sys.modules.keys()):
    if mod_name.startswith('sirius'):
        del sys.modules[mod_name]

print("✅ Sirius installed successfully!")

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Verify installation
import sirius
print(f"✅ Sirius version: {sirius.__version__}")

# Verify RunwareClient has generate method (sanity check)
from sirius._runware_client import RunwareClient
assert hasattr(RunwareClient, 'generate'), "ERROR: RunwareClient.generate method missing! Try restarting kernel."
print("✅ RunwareClient.generate method verified")
print("✅ Asyncio nested loops enabled.")

## 2. API Keys
Sirius requires:
1. **Anthropic API Key** (for Claude Vision 'Director')
2. **Runware API Key** (for Image Generation)

In [ ]:
import os
from getpass import getpass

# Helper to set env vars safely
def set_key(name, prompt_text):
    if not os.environ.get(name):
        key = getpass(prompt_text)
        if key:
            os.environ[name] = key
        else:
            print(f"⚠️ {name} not set.")

print("Please enter your API keys (press Enter to skip if already set in environment):")

set_key("ANTHROPIC_API_KEY", "Enter Anthropic API Key (sk-ant-...): ")
set_key("RUNWARE_API_KEY", "Enter Runware API Key: ")

print("\n✅ Setup complete.")

## 3. Prepare Images
We need a **Start Image** and an **End Image**. You can upload your own to the file browser on the left, or run the cell below to download sample assets.

In [ ]:
import requests
from PIL import Image as PILImage
from io import BytesIO
from IPython.display import display, Image as IPyImage

def download_image(url, filename):
    headers = {
        'User-Agent': 'SiriusDemo/1.0 (https://github.com/MarsZDF/bfl-hack-sirius)'
    }
    try:
        print(f"Downloading {url}...")
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        img = PILImage.open(BytesIO(response.content))
        # Convert to RGB if necessary (e.g. RGBA pngs)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(filename)
        return filename
    except Exception as e:
        print(f"❌ Failed to download {url}: {e}")
        # Fallback: Create a placeholder image if download fails
        print("Creating placeholder image...")
        img = PILImage.new('RGB', (800, 600), color = (73, 109, 137))
        img.save(filename)
        return filename

# Create assets folder
os.makedirs("assets", exist_ok=True)

# Sample images (Use original full-res URLs to avoid thumbnail 429s)
url_a = "https://upload.wikimedia.org/wikipedia/commons/4/4d/Cat_November_2010-1a.jpg"
url_b = "https://upload.wikimedia.org/wikipedia/commons/9/90/Labrador_Retriever_portrait.jpg"

image_a = "assets/start.png"
image_b = "assets/end.png"

if not os.path.exists(image_a):
    download_image(url_a, image_a)

if not os.path.exists(image_b):
    download_image(url_b, image_b)

print(f"Start: {image_a}")
print(f"End:   {image_b}")

if os.path.exists(image_a):
    display(IPyImage(filename=image_a, width=300))
if os.path.exists(image_b):
    display(IPyImage(filename=image_b, width=300))

## 4. Configure Your Morph

Choose your transition style and number of frames. More frames = longer video (at 8fps).

**Available styles:**
- **METAMORPHOSIS** - Biological transformation (caterpillar→butterfly) ⭐ Recommended
- **MORPH** - Direct visual interpolation
- **TIMELAPSE** - Time passing (always positive outcomes!)
- **GLITCH** - Digital corruption aesthetic
- **PAINTERLY** - Art style evolution (photo→impressionist→cubist)
- **SURREAL** - Dreamlike transitions
- **NARRATIVE** - Story-driven
- **FADE** - Gradual blend

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from sirius import morph, plan_morph, TransitionStyle, AspectRatio

# --- Interactive Configuration ---
# Preserve existing values if widgets already exist (avoid reset on re-run)

# User context text (FIRST - so you can type before selecting style)
_prev_context = context_input.value if 'context_input' in dir() else ''
context_input = widgets.Text(
    value=_prev_context,
    placeholder='e.g., "Make it cyberpunk with neon colors"',
    description='Context:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='450px')
)

# Style dropdown - preserve previous selection
_prev_style = style_dropdown.value if 'style_dropdown' in dir() else 'metamorphosis'
style_dropdown = widgets.Dropdown(
    options=[
        ('Metamorphosis (Recommended)', 'metamorphosis'),
        ('Morph', 'morph'),
        ('Timelapse', 'timelapse'),
        ('Glitch', 'glitch'),
        ('Painterly', 'painterly'),
        ('Surreal', 'surreal'),
        ('Narrative', 'narrative'),
        ('Fade', 'fade'),
    ],
    value=_prev_style,
    description='Style:',
    style={'description_width': '80px'}
)

# Aspect ratio dropdown - preserve previous selection
_prev_aspect = aspect_dropdown.value if 'aspect_dropdown' in dir() else '9:16'
aspect_dropdown = widgets.Dropdown(
    options=[
        ('9:16 - TikTok/Reels (Recommended)', '9:16'),
        ('16:9 - YouTube/Desktop', '16:9'),
        ('1:1 - Instagram Feed', '1:1'),
        ('21:9 - Cinematic', '21:9'),
        ('4:3 - Classic Photo', '4:3'),
    ],
    value=_prev_aspect,
    description='Aspect:',
    style={'description_width': '80px'}
)

# Frame count slider - preserve previous value
_prev_frames = frame_slider.value if 'frame_slider' in dir() else 16
frame_slider = widgets.IntSlider(
    value=_prev_frames,
    min=4,
    max=24,
    step=1,
    description='Frames:',
    style={'description_width': '80px'},
    readout=True,
    readout_format='d'
)

# Duration display (updates based on frames)
duration_label = widgets.Label(value=f'Video duration: ~{_prev_frames/8:.1f}s (at 8fps)')

def update_duration(change):
    duration = change['new'] / 8  # 8fps
    duration_label.value = f'Video duration: ~{duration:.1f}s (at 8fps)'

frame_slider.observe(update_duration, names='value')

# Display widgets in order: context first!
print("🎬 Configure your morph:")
print("   1. Add creative direction (optional)")
print("   2. Choose style, aspect ratio, and frame count")
print("   3. Run the next cell to generate\n")
display(context_input)
display(style_dropdown)
display(aspect_dropdown)
display(frame_slider)
display(duration_label)

In [ ]:
# --- Generate Preview ---
# This cell uses the settings from the widgets above

from sirius.pipeline import morph
from sirius import TransitionStyle, AspectRatio, GenerationConfig
from IPython.display import display, Image as IPyImage

# Get values from widgets (read fresh each time)
raw_style_value = style_dropdown.value
selected_style = TransitionStyle(raw_style_value)
raw_aspect_value = aspect_dropdown.value
selected_aspect = AspectRatio(raw_aspect_value)
width, height = selected_aspect.get_dimensions()
num_frames = frame_slider.value
user_context = context_input.value.strip() or None

print(f"🎬 Configuration:")
print(f"   Style:  {selected_style.value} (dropdown: '{raw_style_value}')")
print(f"   Aspect: {raw_aspect_value} ({width}x{height})")
print(f"   Frames: {num_frames}")
print(f"   Context: {user_context or '(none)'}")
print()

# Create generation config with aspect ratio dimensions
gen_config = GenerationConfig(width=width, height=height)

# 1. Plan the transition
print("📋 Planning transition with Claude Vision...")
plan = plan_morph(
    image_a,
    image_b,
    frame_count=num_frames,
    transition_style=selected_style,
    user_context=user_context
)

# Show first few prompts
print("\n📝 Frame prompts (first 3):")
for i, p in enumerate(plan.prompts[:3]):
    print(f"   {i+1}: {p[:80]}...")
if len(plan.prompts) > 3:
    print(f"   ... and {len(plan.prompts) - 3} more")

# 2. Generate Preview
print("\n⚡ Generating preview (fast mode)...")
preview_result = morph(
    image_a,
    image_b,
    plan=plan,
    preview=True,
    output_dir="outputs/preview",
    config=gen_config
)

print(f"\n✅ Preview saved: {preview_result.video_path}")
display(IPyImage(filename=preview_result.video_path, width=800))

from IPython.display import Video

def on_progress(update):
    print(f"{update.progress:.0%} - {update.message}")

print(f"🎬 Generating full video ({num_frames} frames, {selected_style.value} style)...")
print("This might take 2-3 minutes...\n")

result = morph(
    image_a,
    image_b,
    plan=plan,     # Reuse the plan from preview
    preview=False, # High quality mode
    boomerang=False, # Linear A -> B (Set to True for loop)
    on_progress=on_progress,
    output_dir="outputs/final"
)

print(f"\n✅ Video saved: {result.video_path}")
display(Video(result.video_path, embed=True, width=600))

In [ ]:
from IPython.display import Video

def on_progress(update):
    print(f"{update.progress:.0%} - {update.message}")

print(f"🎬 Generating full video ({num_frames} frames, {selected_style.value} style, {raw_aspect_value})...")
print("This might take 2-3 minutes...\n")

result = morph(
    image_a,
    image_b,
    plan=plan,     # Reuse the plan from preview
    preview=False, # High quality mode
    boomerang=False, # Linear A -> B (Set to True for loop)
    on_progress=on_progress,
    output_dir="outputs/final",
    config=gen_config  # Use same aspect ratio config
)

print(f"\n✅ Video saved: {result.video_path}")
display(Video(result.video_path, embed=True, width=600))

# Quick gallery: Try 3 different styles with 4-frame previews
gallery_styles = [
    s for s in [TransitionStyle.GLITCH, TransitionStyle.TIMELAPSE, TransitionStyle.PAINTERLY]
    if s != selected_style  # Skip the style we already used
][:3]

for style in gallery_styles:
    print(f"\n🎨 Previewing: {style.value.upper()}")
    try:
        p_result = morph(
            image_a, 
            image_b,
            frame_count=4,  # Quick 4-frame preview
            transition_style=style,
            preview=True,
            output_dir=f"outputs/style_{style.value}",
            user_context=user_context
        )
        display(IPyImage(filename=p_result.video_path, width=600))
    except Exception as e:
        print(f"   Skipped: {e}")

In [ ]:
# Try a few interesting styles (skip METAMORPHOSIS since we already used it)
styles = [TransitionStyle.GLITCH, TransitionStyle.TIMELAPSE, TransitionStyle.PAINTERLY]

for style in styles:
    print(f"\n🎨 Previewing Style: {style.value.upper()}")
    try:
        p_result = morph(
            image_a, 
            image_b,
            frame_count=4, # Just 4 frames for speed
            transition_style=style,
            preview=True, # Fast Flux Schnell
            output_dir=f"outputs/style_{style.value}",
            user_context=user_context # Pass the same context
        )
        display(IPyImage(filename=p_result.video_path, width=600))
    except Exception as e:
        print(f"Skipping {style}: {e}")